In [3]:
import pandas as pd

In [4]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 

In [5]:
raw_df = pd.read_html("https://en.wikipedia.org/wiki/Academy_Award_for_Best_Director")[2]

In [6]:
director_df = raw_df[raw_df["Year"]!=raw_df["Film"]]

In [7]:
list_of_years = director_df.Year.unique()

In [8]:
years = []
director=[]
film=[]
for x in list_of_years:
    years.append(director_df[director_df["Year"]==x].iloc[0]["Year"])
    director.append(director_df[director_df["Year"]==x].iloc[0]["Director(s)"])
    film.append(director_df[director_df["Year"]==x].iloc[0]["Film"])

In [9]:
bestdirector_df = pd.DataFrame({"Years": years, "Winner Name":director, "Film": film})

In [10]:
bestdirector_df

,Years,Winner Name,Film
0,1927/28 (1st),Frank Borzage (Dramatic Picture),7th Heaven
1,1928/1929(2nd)[note 1],Frank Lloyd,The Divine Lady
2,1929/1930(3rd),Lewis Milestone,All Quiet on the Western Front
3,1930/1931(4th),Norman Taurog,Skippy
4,1931/1932(5th),Frank Borzage,Bad Girl
...,...,...,...
87,2015 (88th),Alejandro G. Iñárritu,The Revenant
88,2016 (89th),Damien Chazelle,La La Land
89,2017 (90th),Guillermo del Toro,The Shape of Water
90,2018 (91st),Alfonso Cuarón,Roma


In [11]:
directors = []

for i in director:
    directors.append(i.split("&"))
    

In [12]:
directors

[['Frank Borzage (Dramatic Picture)'],
 ['Frank Lloyd'],
 ['Lewis Milestone'],
 ['Norman Taurog'],
 ['Frank Borzage'],
 ['Frank Lloyd'],
 ['Frank Capra'],
 ['John Ford'],
 ['Frank Capra'],
 ['Leo McCarey'],
 ['Frank Capra'],
 ['Victor Fleming'],
 ['John Ford'],
 ['John Ford'],
 ['William Wyler'],
 ['Michael Curtiz'],
 ['Leo McCarey'],
 ['Billy Wilder'],
 ['William Wyler'],
 ['Elia Kazan'],
 ['John Huston'],
 ['Joseph L. Mankiewicz'],
 ['Joseph L. Mankiewicz'],
 ['George Stevens'],
 ['John Ford'],
 ['Fred Zinnemann'],
 ['Elia Kazan'],
 ['Delbert Mann'],
 ['George Stevens'],
 ['David Lean'],
 ['Vincente Minnelli'],
 ['William Wyler'],
 ['Billy Wilder'],
 ['Robert Wise ', ' Jerome Robbins'],
 ['David Lean'],
 ['Tony Richardson'],
 ['George Cukor'],
 ['Robert Wise'],
 ['Fred Zinnemann'],
 ['Mike Nichols'],
 ['Carol Reed'],
 ['John Schlesinger'],
 ['Franklin J. Schaffner'],
 ['William Friedkin'],
 ['Bob Fosse'],
 ['George Roy Hill'],
 ['Francis Ford Coppola'],
 ['Miloš Forman'],
 ['John G. 